# Compute performance metrics for the given Y and Y_score without sklearn

In [2]:
import numpy as np
import pandas as pd
# other than these two you should not import any other packages

<pre>
<font color='red'><b>A.</b></font> Compute performance metrics for the given data <strong>5_a.csv</strong>
   <b>Note 1:</b> in this data you can see number of positive points >> number of negatives points
   <b>Note 2:</b> use pandas or numpy to read the data from <b>5_a.csv</b>
   <b>Note 3:</b> you need to derive the class labels from given score</pre> $y^{pred}= \text{[0 if y_score < 0.5 else 1]}$

<pre>
<ol>
<li> Compute Confusion Matrix </li>
<li> Compute F1 Score </li>
<li> Compute AUC Score, you need to compute different thresholds and for each threshold compute tpr,fpr and then use               numpy.trapz(tpr_array, fpr_array) <a href='https://stackoverflow.com/q/53603376/4084039'>https://stackoverflow.com/q/53603376/4084039</a>, <a href='https://stackoverflow.com/a/39678975/4084039'>https://stackoverflow.com/a/39678975/4084039</a> Note: it should be numpy.trapz(tpr_array, fpr_array) not numpy.trapz(fpr_array, tpr_array)</li>
<li> Compute Accuracy Score </li>
</ol>
</pre>

In [ ]:
# write your code here
dataset_A=pd.read_csv("/content/5_a.csv")

In [ ]:
dataset_A.head()

,y,proba
0,1.0,0.637387
1,1.0,0.635165
2,1.0,0.766586
3,1.0,0.724564
4,1.0,0.889199


In [ ]:
#Let's create a third column for y_pred
dataset_A["y_hat"]=dataset_A["proba"].apply( lambda x: 1 if x>0.5 else 0 )

In [ ]:
dataset_A.columns

Index(['y', 'proba', 'y_hat'], dtype='object')

In [ ]:
dataset_A.iloc[:,2].sum()

10100

In [ ]:
#Let's create a function called for confusion matrix
def metrics(dataset):
  TP=TN=FP=FN=0
  TP = ((dataset.iloc[:,0]==1.0) & (dataset.iloc[:,2] == 1.0)).sum()  
  TN = ((dataset.iloc[:,0]==0.0) & (dataset.iloc[:,2] == 0.0)).sum()
  FP = ((dataset.iloc[:,0]==0.0) & (dataset.iloc[:,2] == 1.0)).sum()
  FN = ((dataset.iloc[:,0]==1.0) & (dataset.iloc[:,2] == 0.0)).sum()

  confusion_matrix=[[TP,FP],[FN,TN]]
  precision=TP/(TP+FP)
  recall=TP/(TP+FN)
  FPR=(FP)/(FP+TN)
  Accuracy=(TP+TN)/(TP+FP+FN+TN)
  return  confusion_matrix,precision,recall,Accuracy,FPR

In [ ]:
ConfusionMat,Pre,Re,Acc,FPR=metrics(dataset_A)

In [ ]:
#1.Confusion Matrix 
print(ConfusionMat)

[[10000, 100], [0, 0]]


In [ ]:
#2.F-1 Score
print((2*Pre*Re)/(Pre+Re))

0.9950248756218906


In [ ]:
#4.Accuracy
print(Acc)

0.9900990099009901


In [ ]:
dataset_A.drop(['y_hat'],axis=1,inplace=True)

In [5]:
from tqdm import tqdm

In [ ]:
def AUC_Score(dataset):
  dataset.sort_values(by=dataset.columns[1], axis=0, ascending=False, inplace=True)
  TPR_Array=[]
  FPR_Array=[]
  for threshold in tqdm(dataset.iloc[:,1].unique()):
       dataset['y_hat'] =np.where( dataset.iloc[:,1] >=threshold,1,0) 
       confusion_matrix,precision,recall,Accuracy,FPR= metrics(dataset)
       TPR_Array.append(recall)
       FPR_Array.append(FPR)
       dataset.drop(["y_hat"],axis=1,inplace=True) 
  return np.trapz(TPR_Array,FPR_Array)     

In [ ]:
AUC_Score(dataset_A)

100%|██████████| 10100/10100 [01:08<00:00, 147.61it/s]


0.48829900000000004

<pre>
<font color='red'><b>B.</b></font> Compute performance metrics for the given data <strong>5_b.csv</strong>
   <b>Note 1:</b> in this data you can see number of positive points << number of negatives points
   <b>Note 2:</b> use pandas or numpy to read the data from <b>5_b.csv</b>
   <b>Note 3:</b> you need to derive the class labels from given score</pre> $y^{pred}= \text{[0 if y_score < 0.5 else 1]}$

<pre>
<ol>
<li> Compute Confusion Matrix </li>
<li> Compute F1 Score </li>
<li> Compute AUC Score, you need to compute different thresholds and for each threshold compute tpr,fpr and then use               numpy.trapz(tpr_array, fpr_array) <a href='https://stackoverflow.com/q/53603376/4084039'>https://stackoverflow.com/q/53603376/4084039</a>, <a href='https://stackoverflow.com/a/39678975/4084039'>https://stackoverflow.com/a/39678975/4084039</a></li>
<li> Compute Accuracy Score </li>
</ol>
</pre>

In [ ]:
# write your code
dataset_B=pd.read_csv("/content/5_b.csv")

In [ ]:
dataset_B["y_hat"]=dataset_B["proba"].apply( lambda x: 1 if x>0.5 else 0 )

In [ ]:
ConfusionMat,Pre,Re,Acc,FPR=metrics(dataset_B)

In [ ]:
#ConfusionMatrix
print(ConfusionMat)

[[55, 239], [45, 9761]]


In [ ]:
#F-1 Score
print((2*Pre*Re)/(Pre+Re))

0.2791878172588833


In [ ]:
#Accuracy
print(Acc)

0.9718811881188119


In [ ]:
dataset_B.drop(['y_hat'],axis=1,inplace=True)

In [ ]:
#AUC_Score
AUC_Score(dataset_B)

100%|██████████| 10100/10100 [01:08<00:00, 146.84it/s]


0.9377570000000001

<font color='red'><b>C.</b></font> Compute the best threshold (similarly to ROC curve computation) of probability which gives lowest values of metric <b>A</b> for the given data <strong>5_c.csv</strong>
<br>

you will be predicting label of a data points like this: $y^{pred}= \text{[0 if y_score < threshold  else 1]}$

$ A = 500 \times \text{number of false negative} + 100 \times \text{numebr of false positive}$

<pre>
   <b>Note 1:</b> in this data you can see number of negative points > number of positive points
   <b>Note 2:</b> use pandas or numpy to read the data from <b>5_c.csv</b>
</pre>

In [3]:
 # write your code
 dataset_C=pd.read_csv("/content/5_c.csv")

In [13]:
dataset_C.sort_values(by=dataset_C.columns[1], axis=0, ascending=False, inplace=True)
lowestnumber=0
lowestvalueyielding_threshold=0 
for threshold in tqdm(dataset_C.iloc[:,1].unique()):
   dataset_C['y_hat'] =np.where( dataset_C.iloc[:,1] >=threshold,0,1) 
   FP = ((dataset_C.iloc[:,0]==0.0) & (dataset_C.iloc[:,2] == 1.0)).sum()
   FN = ((dataset_C.iloc[:,0]==1.0) & (dataset_C.iloc[:,2] == 0.0)).sum()
   if lowestnumber<(500*FN+100*FP):
      lowestnumber=(500*FN+100*FP)
      lowestvalueyielding_threshold=threshold
   dataset_C.drop(["y_hat"],axis=1,inplace=True)


100%|██████████| 2791/2791 [00:09<00:00, 308.39it/s]


In [14]:
print("Threshold at which the equation attains minimum is",lowestvalueyielding_threshold)

Threshold at which the equation attains minimum is 0.2300390278970873


<pre>
<font color='red'><b>D.</b></font> Compute performance metrics(for regression) for the given data <strong>5_d.csv</strong>
    <b>Note 2:</b> use pandas or numpy to read the data from <b>5_d.csv</b>
    <b>Note 1:</b> <b>5_d.csv</b> will having two columns Y and predicted_Y both are real valued features
<ol>
<li> Compute Mean Square Error </li>
<li> Compute MAPE: https://www.youtube.com/watch?v=ly6ztgIkUxk</li>
<li> Compute R^2 error: https://en.wikipedia.org/wiki/Coefficient_of_determination#Definitions </li>
</ol>
</pre>

In [ ]:
dataset_D=pd.read_csv("/content/5_d.csv")

In [ ]:
dataset_D.columns

Index(['y', 'pred'], dtype='object')

In [ ]:
dataset_D["Sq_Diff"]=(dataset_D["y"]-dataset_D["pred"])**2/(len(dataset_D))

In [ ]:
#1.Mean Square Error
np.sum(dataset_D["Sq_Diff"])

177.16569974554702

In [ ]:
#2.MAPE
dataset_D["abs_Diff"]=abs(dataset_D["y"]-dataset_D["pred"])/(np.sum(dataset_D["y"]))

In [ ]:
np.sum(dataset_D["abs_Diff"])

0.12912029940096875

In [ ]:
#3.R^2
dataset_D["SST"]=(dataset_D["pred"]-np.mean(dataset_D["y"]))**2/(len(dataset_D))

In [ ]:
print("R^2 is",1-((np.sum(dataset_D["Sq_Diff"]))/(np.sum(dataset_D["SST"]))))

R^2 is 0.9999667760476615
